# 🌏 Scraper de Changements Réglementaires - Marchés Asiatiques

Ce notebook scrape les changements réglementaires des marchés boursiers de:
- 🇹🇭 **Thaïlande** (SEC Thailand)
- 🇸🇬 **Singapour** (MAS - Monetary Authority of Singapore)
- 🇲🇾 **Malaisie** (SC Malaysia)

## 📋 Méthodologie
- Utilisation de **Selenium** pour les sites dynamiques
- **BeautifulSoup** pour le parsing HTML
- **Pandas** pour l'analyse des données
- Export vers CSV et Excel

In [ ]:
# Import des bibliothèques essentielles
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import re
import warnings
warnings.filterwarnings('ignore')

# Selenium pour le scraping dynamique
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# BeautifulSoup pour le parsing
from bs4 import BeautifulSoup

# Requests pour les requêtes HTTP simples
import requests

print("✅ Bibliothèques importées avec succès!")
print(f"📅 Date d'exécution: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## ⚙️ Configuration du Driver Selenium

In [ ]:
def setup_driver(headless=True):
    """
    Configure le driver Selenium Chrome
    
    Args:
        headless: Si True, exécute le navigateur en mode headless (sans interface)
    
    Returns:
        driver: Instance du WebDriver Chrome
    """
    chrome_options = Options()
    
    if headless:
        chrome_options.add_argument('--headless')
    
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36')
    
    # Désactiver les notifications
    chrome_options.add_experimental_option('prefs', {
        'profile.default_content_setting_values.notifications': 2
    })
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(10)
    
    return driver

print("✅ Fonction de configuration du driver créée")

## 🇹🇭 1. Scraping - SEC Thailand

In [ ]:
def scrape_sec_thailand():
    """
    Scrape les changements réglementaires de la SEC Thailand
    Site: https://www.sec.or.th/EN/Pages/News_Detail.aspx
    """
    print("🇹🇭 Scraping SEC Thailand...")
    
    url = "https://www.sec.or.th/EN/Pages/News_All.aspx"
    
    try:
        # Utiliser requests pour cette page (plus rapide)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=15)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        regulations = []
        
        # Recherche des articles de news/régulations
        news_items = soup.find_all(['div', 'article', 'li'], class_=re.compile('news|item|article', re.I))
        
        if not news_items:
            # Alternative: chercher tous les liens avec dates
            news_items = soup.find_all('a', href=True)
        
        for item in news_items[:30]:  # Limiter aux 30 premiers
            try:
                # Extraction du titre
                title_elem = item.find(['h2', 'h3', 'h4', 'a']) or item
                title = title_elem.get_text(strip=True)
                
                # Filter pour les régulations
                if len(title) < 10 or not any(word in title.lower() for word in 
                    ['regulation', 'rule', 'law', 'act', 'requirement', 'guideline', 'circular']):
                    continue
                
                # Extraction de la date
                date_text = item.get_text()
                date_match = re.search(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{4}[/-]\d{1,2}[/-]\d{1,2}', date_text)
                date = date_match.group(0) if date_match else 'N/A'
                
                # Extraction du lien
                link_elem = item.find('a', href=True) or (item if item.name == 'a' else None)
                link = link_elem['href'] if link_elem else ''
                if link and not link.startswith('http'):
                    link = 'https://www.sec.or.th' + link
                
                regulations.append({
                    'Country': 'Thailand 🇹🇭',
                    'Authority': 'SEC Thailand',
                    'Title': title,
                    'Date': date,
                    'Link': link,
                    'Scraped_At': datetime.now().strftime('%Y-%m-%d %H:%M')
                })
            except Exception as e:
                continue
        
        print(f"✅ SEC Thailand: {len(regulations)} régulations trouvées")
        return regulations
        
    except Exception as e:
        print(f"❌ Erreur SEC Thailand: {e}")
        return []

# Test de la fonction
thailand_data = scrape_sec_thailand()
if thailand_data:
    df_thailand = pd.DataFrame(thailand_data)
    print("\n📊 Aperçu des données Thaïlande:")
    display(df_thailand.head())

## 🇸🇬 2. Scraping - MAS Singapore

In [ ]:
def scrape_mas_singapore():
    """
    Scrape les changements réglementaires de MAS Singapore
    Site: https://www.mas.gov.sg/news
    """
    print("🇸🇬 Scraping MAS Singapore...")
    
    url = "https://www.mas.gov.sg/news/regulations-and-circulars"
    
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=15)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        regulations = []
        
        # Recherche des éléments de régulation
        items = soup.find_all(['div', 'article', 'li'], class_=re.compile('card|item|news|article', re.I))
        
        if not items:
            items = soup.find_all('a', href=re.compile('regulation|circular|notice', re.I))
        
        for item in items[:30]:
            try:
                # Titre
                title_elem = item.find(['h2', 'h3', 'h4', 'span', 'a'])
                if not title_elem:
                    continue
                title = title_elem.get_text(strip=True)
                
                if len(title) < 10:
                    continue
                
                # Date
                date_elem = item.find(['time', 'span'], class_=re.compile('date', re.I))
                if date_elem:
                    date = date_elem.get_text(strip=True)
                else:
                    date_match = re.search(r'\d{1,2}\s+(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s+\d{4}', item.get_text())
                    date = date_match.group(0) if date_match else 'N/A'
                
                # Lien
                link_elem = item.find('a', href=True)
                link = link_elem['href'] if link_elem else ''
                if link and not link.startswith('http'):
                    link = 'https://www.mas.gov.sg' + link
                
                regulations.append({
                    'Country': 'Singapore 🇸🇬',
                    'Authority': 'MAS',
                    'Title': title,
                    'Date': date,
                    'Link': link,
                    'Scraped_At': datetime.now().strftime('%Y-%m-%d %H:%M')
                })
            except:
                continue
        
        print(f"✅ MAS Singapore: {len(regulations)} régulations trouvées")
        return regulations
        
    except Exception as e:
        print(f"❌ Erreur MAS Singapore: {e}")
        return []

# Test de la fonction
singapore_data = scrape_mas_singapore()
if singapore_data:
    df_singapore = pd.DataFrame(singapore_data)
    print("\n📊 Aperçu des données Singapour:")
    display(df_singapore.head())

## 🇲🇾 3. Scraping - SC Malaysia

In [ ]:
def scrape_sc_malaysia():
    """
    Scrape les changements réglementaires de SC Malaysia
    Site: https://www.sc.com.my
    """
    print("🇲🇾 Scraping SC Malaysia...")
    
    url = "https://www.sc.com.my/regulation/guidelines"
    
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=15)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        regulations = []
        
        # Recherche des items
        items = soup.find_all(['div', 'article', 'li', 'tr'], class_=re.compile('item|card|row|news', re.I))
        
        if not items:
            items = soup.find_all('a', href=True)
        
        for item in items[:30]:
            try:
                # Titre
                title_elem = item.find(['h2', 'h3', 'h4', 'td', 'span', 'a'])
                if not title_elem:
                    continue
                title = title_elem.get_text(strip=True)
                
                if len(title) < 10:
                    continue
                
                # Date
                date_elem = item.find(['time', 'span', 'td'], class_=re.compile('date', re.I))
                if date_elem:
                    date = date_elem.get_text(strip=True)
                else:
                    date_match = re.search(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{4}[/-]\d{1,2}[/-]\d{1,2}', item.get_text())
                    date = date_match.group(0) if date_match else 'N/A'
                
                # Lien
                link_elem = item.find('a', href=True)
                link = link_elem['href'] if link_elem else ''
                if link and not link.startswith('http'):
                    link = 'https://www.sc.com.my' + link
                
                regulations.append({
                    'Country': 'Malaysia 🇲🇾',
                    'Authority': 'SC Malaysia',
                    'Title': title,
                    'Date': date,
                    'Link': link,
                    'Scraped_At': datetime.now().strftime('%Y-%m-%d %H:%M')
                })
            except:
                continue
        
        print(f"✅ SC Malaysia: {len(regulations)} régulations trouvées")
        return regulations
        
    except Exception as e:
        print(f"❌ Erreur SC Malaysia: {e}")
        return []

# Test de la fonction
malaysia_data = scrape_sc_malaysia()
if malaysia_data:
    df_malaysia = pd.DataFrame(malaysia_data)
    print("\n📊 Aperçu des données Malaisie:")
    display(df_malaysia.head())

## 🔄 4. Scraping avec Selenium (pour sites dynamiques)

In [ ]:
def scrape_with_selenium(url, country, authority):
    """
    Scraping générique avec Selenium pour les sites nécessitant JavaScript
    """
    print(f"🔄 Scraping {country} avec Selenium...")
    
    driver = None
    try:
        driver = setup_driver(headless=True)
        driver.get(url)
        
        # Attendre le chargement de la page
        time.sleep(3)
        
        # Scroll pour charger le contenu dynamique
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        # Parser le contenu
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        regulations = []
        
        # Recherche flexible d'articles
        items = soup.find_all(['article', 'div', 'li'], 
                             class_=re.compile('news|item|card|article|regulation', re.I))
        
        for item in items[:25]:
            try:
                title_elem = item.find(['h1', 'h2', 'h3', 'h4', 'a'])
                if not title_elem:
                    continue
                    
                title = title_elem.get_text(strip=True)
                if len(title) < 15:
                    continue
                
                # Date
                date_elem = item.find(['time', 'span'], class_=re.compile('date', re.I))
                date = date_elem.get_text(strip=True) if date_elem else 'N/A'
                
                # Lien
                link_elem = item.find('a', href=True)
                link = link_elem['href'] if link_elem else ''
                
                regulations.append({
                    'Country': country,
                    'Authority': authority,
                    'Title': title,
                    'Date': date,
                    'Link': link,
                    'Scraped_At': datetime.now().strftime('%Y-%m-%d %H:%M')
                })
            except:
                continue
        
        print(f"✅ {len(regulations)} éléments trouvés")
        return regulations
        
    except Exception as e:
        print(f"❌ Erreur: {e}")
        return []
    finally:
        if driver:
            driver.quit()

print("✅ Fonction Selenium créée")

## 📊 5. Consolidation et Analyse des Données

In [ ]:
# Consolidation de toutes les données
all_regulations = []

if thailand_data:
    all_regulations.extend(thailand_data)
if singapore_data:
    all_regulations.extend(singapore_data)
if malaysia_data:
    all_regulations.extend(malaysia_data)

# Création du DataFrame consolidé
df_all = pd.DataFrame(all_regulations)

print("="*80)
print("📊 RÉSUMÉ DES DONNÉES COLLECTÉES")
print("="*80)
print(f"\n📈 Total de régulations: {len(df_all)}")
print(f"\n📍 Répartition par pays:")
print(df_all['Country'].value_counts())
print(f"\n🏛️ Répartition par autorité:")
print(df_all['Authority'].value_counts())

print("\n" + "="*80)
print("📋 APERÇU DES DONNÉES")
print("="*80)
display(df_all.head(10))

# Statistiques
print("\n📊 Informations du dataset:")
print(df_all.info())

## 🔍 6. Filtrage et Recherche

In [ ]:
def search_regulations(df, keyword, country=None):
    """
    Recherche dans les régulations par mot-clé
    
    Args:
        df: DataFrame des régulations
        keyword: Mot-clé à rechercher
        country: Pays optionnel pour filtrer
    """
    mask = df['Title'].str.contains(keyword, case=False, na=False)
    
    if country:
        mask &= df['Country'].str.contains(country, case=False, na=False)
    
    results = df[mask]
    
    print(f"🔍 Résultats pour '{keyword}': {len(results)} trouvés")
    return results

# Exemples de recherches
print("📌 Exemple 1: Recherche 'ESG' (Environmental, Social, Governance)")
esg_results = search_regulations(df_all, 'ESG')
if len(esg_results) > 0:
    display(esg_results[['Country', 'Title', 'Date']].head())

print("\n📌 Exemple 2: Recherche 'disclosure'")
disclosure_results = search_regulations(df_all, 'disclosure')
if len(disclosure_results) > 0:
    display(disclosure_results[['Country', 'Title', 'Date']].head())

print("\n📌 Exemple 3: Recherche 'crypto' ou 'digital'")
crypto_mask = df_all['Title'].str.contains('crypto|digital|blockchain', case=False, na=False)
crypto_results = df_all[crypto_mask]
print(f"🔍 {len(crypto_results)} régulations liées aux crypto/digital trouvées")
if len(crypto_results) > 0:
    display(crypto_results[['Country', 'Title', 'Date']].head())

## 📊 7. Visualisations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration du style
plt.style.use('dark_background')
sns.set_palette("husl")

# Création des visualisations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('📊 Analyse des Changements Réglementaires - Marchés Asiatiques', 
             fontsize=16, fontweight='bold', y=1.02)

# 1. Répartition par pays
country_counts = df_all['Country'].value_counts()
axes[0, 0].pie(country_counts.values, labels=country_counts.index, autopct='%1.1f%%',
               colors=['#ff6b6b', '#4ecdc4', '#45b7d1'])
axes[0, 0].set_title('Répartition par Pays', fontweight='bold')

# 2. Nombre par autorité
authority_counts = df_all['Authority'].value_counts()
axes[0, 1].barh(authority_counts.index, authority_counts.values, 
                color=['#ff6b6b', '#4ecdc4', '#45b7d1'])
axes[0, 1].set_xlabel('Nombre de régulations')
axes[0, 1].set_title('Régulations par Autorité', fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# 3. Longueur des titres
df_all['Title_Length'] = df_all['Title'].str.len()
axes[1, 0].hist(df_all['Title_Length'], bins=20, color='#26de81', alpha=0.7, edgecolor='white')
axes[1, 0].set_xlabel('Longueur du titre (caractères)')
axes[1, 0].set_ylabel('Fréquence')
axes[1, 0].set_title('Distribution de la Longueur des Titres', fontweight='bold')
axes[1, 0].grid(alpha=0.3)

# 4. Top mots-clés
import collections
all_words = ' '.join(df_all['Title']).lower()
words = re.findall(r'\b[a-z]{4,}\b', all_words)
word_freq = collections.Counter(words)
common_words = dict(word_freq.most_common(10))

axes[1, 1].barh(list(common_words.keys()), list(common_words.values()),
                color='#ffa502')
axes[1, 1].set_xlabel('Fréquence')
axes[1, 1].set_title('Top 10 Mots-Clés', fontweight='bold')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Visualisations créées avec succès")

## 💾 8. Export des Données

In [ ]:
# Export vers CSV
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_filename = f'regulatory_changes_asia_{timestamp}.csv'
df_all.to_csv(csv_filename, index=False, encoding='utf-8-sig')
print(f"✅ Données exportées vers: {csv_filename}")

# Export vers Excel avec formatage
excel_filename = f'regulatory_changes_asia_{timestamp}.xlsx'
with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
    # Sheet 1: Toutes les données
    df_all.to_excel(writer, sheet_name='All Regulations', index=False)
    
    # Sheet 2: Thailand
    df_thailand = df_all[df_all['Country'].str.contains('Thailand')]
    if len(df_thailand) > 0:
        df_thailand.to_excel(writer, sheet_name='Thailand', index=False)
    
    # Sheet 3: Singapore
    df_singapore = df_all[df_all['Country'].str.contains('Singapore')]
    if len(df_singapore) > 0:
        df_singapore.to_excel(writer, sheet_name='Singapore', index=False)
    
    # Sheet 4: Malaysia
    df_malaysia = df_all[df_all['Country'].str.contains('Malaysia')]
    if len(df_malaysia) > 0:
        df_malaysia.to_excel(writer, sheet_name='Malaysia', index=False)
    
    # Sheet 5: Summary
    summary_data = {
        'Métrique': [
            'Total Régulations',
            'Thailand',
            'Singapore',
            'Malaysia',
            'Date de scraping'
        ],
        'Valeur': [
            len(df_all),
            len(df_thailand),
            len(df_singapore),
            len(df_malaysia),
            datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        ]
    }
    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)

print(f"✅ Données exportées vers: {excel_filename}")

print("\n" + "="*80)
print("✅ EXPORT TERMINÉ")
print("="*80)
print(f"📁 Fichiers créés:")
print(f"   - {csv_filename}")
print(f"   - {excel_filename}")

## 🔄 9. Fonction de Scraping Complète (One-Click)

In [ ]:
def scrape_all_markets(export=True):
    """
    Fonction principale pour scraper tous les marchés d'un coup
    
    Args:
        export: Si True, exporte automatiquement les résultats
    
    Returns:
        DataFrame consolidé de toutes les régulations
    """
    print("🚀 Démarrage du scraping complet...")
    print("="*80)
    
    all_data = []
    
    # Thailand
    try:
        th_data = scrape_sec_thailand()
        all_data.extend(th_data)
        time.sleep(2)
    except Exception as e:
        print(f"⚠️ Erreur Thailand: {e}")
    
    # Singapore
    try:
        sg_data = scrape_mas_singapore()
        all_data.extend(sg_data)
        time.sleep(2)
    except Exception as e:
        print(f"⚠️ Erreur Singapore: {e}")
    
    # Malaysia
    try:
        my_data = scrape_sc_malaysia()
        all_data.extend(my_data)
    except Exception as e:
        print(f"⚠️ Erreur Malaysia: {e}")
    
    # Création du DataFrame
    df = pd.DataFrame(all_data)
    
    print("="*80)
    print(f"✅ Scraping terminé: {len(df)} régulations collectées")
    
    # Export automatique
    if export and len(df) > 0:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f'asia_regulations_{timestamp}.csv'
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"💾 Données exportées: {filename}")
    
    return df

print("✅ Fonction de scraping complète créée")
print("💡 Utilisez: df = scrape_all_markets() pour lancer le scraping complet")

## 📝 10. Notes et Améliorations Possibles

### ✅ Fonctionnalités actuelles:
- Scraping de 3 marchés asiatiques (TH, SG, MY)
- Support de Selenium pour sites dynamiques
- Export CSV et Excel
- Recherche par mots-clés
- Visualisations

### 🚀 Améliorations possibles:
1. **Scheduling automatique**: Utiliser `schedule` pour scraper quotidiennement
2. **Base de données**: Stocker dans SQLite/PostgreSQL pour historique
3. **Notifications**: Email/Slack quand nouvelles régulations détectées
4. **NLP**: Analyse de sentiment et extraction d'entités
5. **Plus de marchés**: Hong Kong, Japon, Corée du Sud
6. **API REST**: Exposer les données via Flask/FastAPI
7. **Dashboard**: Streamlit ou Dash pour visualisation interactive

### ⚠️ Notes importantes:
- Respecter les `robots.txt` des sites
- Ajouter des delays entre requêtes (rate limiting)
- Vérifier régulièrement si la structure HTML a changé
- Utiliser des proxies pour éviter les blocages IP